In [163]:
import pandas as pd
import yfinance as yf
import numpy as np
import statsmodels.api as sm

In [164]:
SPY_data = yf.download(tickers = '^GSPC', start = '2020-01-01', interval = '60m')
SPY_data['Adj Close'].isna().sum()

[*********************100%***********************]  1 of 1 completed


0

In [165]:
VOO_data = yf.download(tickers = 'VOO', start = '2020-01-01', interval = '60m')
VOO_data['Adj Close'].isna().sum()

[*********************100%***********************]  1 of 1 completed


0

In [166]:
SPLG_data = yf.download(tickers = 'SPLG', start = '2020-01-01', interval = '60m')
SPLG_data['Adj Close'].isna().sum()

[*********************100%***********************]  1 of 1 completed


0

In [167]:
IVV_data = yf.download(tickers = 'IVV', start = '2020-01-01', interval = '60m')
IVV_data['Adj Close'].isna().sum()

[*********************100%***********************]  1 of 1 completed


0

In [168]:
def checkIncreasing(df, n):
    for i in range(n - 1):
        if df.iloc[i] > df.iloc[i + 1]:
            return False
        
    return True

In [169]:
def checkDecreasing(df, n):
    for i in range(n - 1):
        if df.iloc[i] < df.iloc[i + 1]:
            return False
        
    return True

In [170]:
def checkPositive(df, n):
    positive = df.iloc[0] >= 0
    if positive:
        for i in range(1, n):
            if df[i] < 0:
                return False
    return True

In [171]:
def checkNegative(df, n):
    negative = df.iloc[0] < 0
    if negative:
        for i in range(1, n):
            if df[i] > 0:
                return False
            
    return True

In [172]:
stock_df = pd.DataFrame(columns = ['SPY', 'VOO', 'SPLG', 'IVV'], index = SPY_data['2020-05-01':].index[4:])

In [173]:
SPY_Moment = (SPY_data['Adj Close']['2020-05-01':] - SPY_data['Adj Close'].shift(6)).dropna()
SPY_MAVG = SPY_data['Adj Close']['2020-05-01':].rolling(6, min_periods = 1).mean().dropna()
margins = 20000
currAmount = 20000
holdings = 0
actions = []
totalTransactionCost = 0
for i, date in enumerate(SPY_Moment['2020-05-01':].index[4:]):
    currAmount = holdings * SPY_data['Adj Close'][date] + margins
    actions.append('hold')
    if holdings == 0:
        if checkIncreasing(SPY_MAVG.iloc[i: i + 4], 4):
            if checkPositive(SPY_Moment[i: i + 4], 4):
                if margins >= currAmount * 0.02:
                    holdings += currAmount * 0.02 / SPY_data['Adj Close'][date]
                    margins -= currAmount * 0.02 * 1.01
                    totalTransactionCost += currAmount * 0.02 * 0.01
        if checkDecreasing(SPY_MAVG.iloc[i: i + 4], 4):
            if checkNegative(SPY_Moment[i: i + 4], 4):
                holdings -= currAmount * 0.02 / SPY_data['Adj Close'][date]
                margins += currAmount * 0.02 * 0.99
                totalTransactionCost += currAmount * 0.02 * 0.01
    else:
        if holdings < 0:
            if checkIncreasing(SPY_MAVG.iloc[i: i + 4], 4):
                if checkPositive(SPY_Moment[i: i + 4], 4):
                    margins -= holdings * SPY_data['Adj Close'][date] * 1.01
                    holdings = 0
                    totalTransactionCost += currAmount * 0.02 * 0.01
        elif holdings > 0:
            if checkDecreasing(SPY_MAVG.iloc[i: i + 4], 4):
                if checkNegative(SPY_Moment[i: i + 4], 4):
                    margins += holdings * SPY_data['Adj Close'][date] * 0.99
                    holdings = 0
                    totalTransactionCost += currAmount * 0.02 * 0.01
    SPY_currAmount = holdings * SPY_data['Adj Close'][date] + margins
    stock_df.loc[date, 'SPY'] = currAmount
    

In [174]:
IVV_Moment = (IVV_data['Adj Close']['2020-05-01':] - IVV_data['Adj Close'].shift(6)).dropna()
IVV_MAVG = IVV_data['Adj Close']['2020-05-01':].rolling(6, min_periods = 1).mean().dropna()

margins = 20000
currAmount = 20000
holdings = 0
for i, date in enumerate(IVV_Moment['2020-05-01':].index[4:]):
    currAmount = holdings * IVV_data['Adj Close'][date] + margins
    if holdings == 0:
        if checkIncreasing(IVV_MAVG.iloc[i: i + 4], 4):
            if checkPositive(IVV_Moment[i: i + 4], 4):
                if margins >= currAmount * 0.02:
                    holdings += currAmount * 0.02 / IVV_data['Adj Close'][date]
                    margins -= currAmount * 0.02 * 1.01
                    totalTransactionCost += currAmount * 0.02 * 0.01
        if checkDecreasing(IVV_MAVG.iloc[i: i + 4], 4):
            if checkNegative(IVV_Moment[i: i + 4], 4):
                holdings -= currAmount * 0.02 / IVV_data['Adj Close'][date]
                margins += currAmount * 0.02 * 0.99
                totalTransactionCost += currAmount * 0.02 * 0.01
    else:
        if holdings < 0:
            if checkIncreasing(IVV_MAVG.iloc[i: i + 4], 4):
                if checkPositive(IVV_Moment[i: i + 4], 4):
                    margins -= holdings * IVV_data['Adj Close'][date] * 1.01
                    holdings = 0  
                    totalTransactionCost += currAmount * 0.02 * 0.01
        elif holdings > 0:
            if checkDecreasing(IVV_MAVG.iloc[i: i + 4], 4):
                if checkNegative(IVV_Moment[i: i + 4], 4):
                    margins += holdings * IVV_data['Adj Close'][date] * 0.99
                    holdings = 0
                    totalTransactionCost += currAmount * 0.02 * 0.01
    
    currAmount = holdings * IVV_data['Adj Close'][date] + margins
    stock_df.loc[date, 'IVV'] = currAmount

In [175]:
SPLG_Moment = (SPLG_data['Adj Close']['2020-05-01':] - SPLG_data['Adj Close'].shift(6)).dropna()
SPLG_MAVG = SPLG_data['Adj Close']['2020-05-01':].rolling(6, min_periods = 1).mean().dropna()

margins = 20000
currAmount = 20000
holdings = 0
SPLG_currAmounts = []
for i, date in enumerate(SPLG_Moment['2020-05-01':].index[4:]):
    currAmount = holdings * SPLG_data['Adj Close'][date] + margins
    if holdings == 0:
        if checkIncreasing(SPLG_MAVG.iloc[i: i + 4], 4):
            if checkPositive(SPLG_Moment[i: i + 4], 4):
                if margins >= currAmount * 0.02:
                    holdings += currAmount * 0.02 / SPLG_data['Adj Close'][date]
                    margins -= currAmount * 0.02 * 1.01
                    totalTransactionCost += currAmount * 0.02 * 0.01
        if checkDecreasing(SPLG_MAVG.iloc[i: i + 4], 4):
            if checkNegative(SPLG_Moment[i: i + 4], 4):
                holdings -= currAmount * 0.02 / SPLG_data['Adj Close'][date]
                margins += currAmount * 0.02 * 0.99
                totalTransactionCost += currAmount * 0.02 * 0.01
    else:
        if holdings < 0:
            if checkIncreasing(SPLG_MAVG.iloc[i: i + 4], 4):
                if checkPositive(SPLG_Moment[i: i + 4], 4):
                    margins -= holdings * SPLG_data['Adj Close'][date] * 1.01
                    holdings = 0 
                    totalTransactionCost += currAmount * 0.02 * 0.01
        elif holdings > 0:
            if checkDecreasing(SPLG_MAVG.iloc[i: i + 4], 4):
                if checkNegative(SPLG_Moment[i: i + 4], 4):
                    margins += holdings * SPLG_data['Adj Close'][date] * 0.99
                    holdings = 0
                    totalTransactionCost += currAmount * 0.02 * 0.01
    
    currAmount = holdings * SPLG_data['Adj Close'][date] + margins
    stock_df.loc[date, 'SPLG'] = currAmount

In [176]:
VOO_Moment = (VOO_data['Adj Close']['2020-05-01':] - VOO_data['Adj Close'].shift(6)).dropna()
VOO_MAVG = VOO_data['Adj Close']['2020-05-01':].rolling(6, min_periods = 1).mean().dropna()

margins = 20000
currAmount = 20000
holdings = 0

for i, date in enumerate(VOO_Moment['2020-05-01':].index[4:]):
    currAmount = holdings * VOO_data['Adj Close'][date] + margins
    if holdings == 0:
        if checkIncreasing(VOO_MAVG.iloc[i: i + 4], 4):
            if checkPositive(VOO_Moment[i: i + 4], 4):
                if margins >= currAmount * 0.02:
                    holdings += currAmount * 0.02 / VOO_data['Adj Close'][date]
                    margins -= currAmount * 0.02 * 1.01
                    totalTransactionCost += currAmount * 0.02 * 0.01
        if checkDecreasing(VOO_MAVG.iloc[i: i + 4], 4):
            if checkNegative(VOO_Moment[i: i + 4], 4):
                holdings -= currAmount * 0.02 / VOO_data['Adj Close'][date]
                margins += currAmount * 0.02 * 0.99
                totalTransactionCost += currAmount * 0.02 * 0.01
    else:
        if holdings < 0:
            if checkIncreasing(VOO_MAVG.iloc[i: i + 4], 4):
                if checkPositive(VOO_Moment[i: i + 4], 4):
                    margins -= holdings * VOO_data['Adj Close'][date] * 1.01
                    holdings = 0
                    totalTransactionCost += currAmount * 0.02 * 0.01
        elif holdings > 0:
            if checkDecreasing(VOO_MAVG.iloc[i: i + 4], 4):
                if checkNegative(VOO_Moment[i: i + 4], 4):
                    margins += holdings * VOO_data['Adj Close'][date] * 0.99
                    holdings = 0
                    totalTransactionCost += currAmount * 0.02 * 0.01
    
    currAmount = holdings * VOO_data['Adj Close'][date] + margins
    stock_df.loc[date, 'VOO'] = currAmount


In [177]:
net_profit = stock_df.iloc[-1].sum() - 100000
net_return = net_profit / 100000
net_profit, net_return

(5044106.768894653, 50.441067688946525)

In [178]:
sum_port = stock_df.sum(axis = 1)

In [179]:
sum_port.std()

1313162.7278974378

In [181]:
hourly_return = ((sum_port - (sum_port.shift(1))) / sum_port).dropna()
hourly_return

Datetime
2020-05-01 14:30:00-04:00   -5.299924e-05
2020-05-01 15:30:00-04:00    5.025693e-06
2020-05-04 09:30:00-04:00    1.037046e-04
2020-05-04 10:30:00-04:00    1.282741e-05
2020-05-04 11:30:00-04:00   -5.955834e-05
                                 ...     
2021-11-24 15:30:00-05:00    9.818101e-07
2021-11-26 09:30:00-05:00   -3.605458e-04
2021-11-26 10:30:00-05:00   -8.681644e-05
2021-11-26 11:30:00-05:00    7.361889e-05
2021-11-26 13:00:00-05:00   -2.427347e-04
Length: 2772, dtype: float64

In [182]:
hourly_return.std() * np.sqrt(252 * 7)

0.24074903096268455

In [183]:
net_profit / sum_port.std()

3.841189413722542

In [184]:
risk_return = (net_return * 4 - 0.012) / (hourly_return.std() * np.sqrt(252 * 7))
risk_return

838.0190356282562

In [185]:
(net_return - 0.012 / 4) / hourly_return.std() / np.sqrt(63 * 7)

419.0095178141281

In [186]:
vol = hourly_return.std() * np.sqrt(252 * 7)
vol

0.24074903096268455

In [188]:
SPY_hourly = ((SPY_data - SPY_data.shift(1)) / SPY_data).dropna()
SPY_hourly

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2020-01-02 10:30:00-05:00,-0.002324,-0.002007,0.000056,0.000651,0.000651,1.000000
2020-01-02 11:30:00-05:00,0.000626,-0.000278,-0.000374,-0.000083,-0.000083,-0.221765
2020-01-02 12:30:00-05:00,-0.000099,0.000462,0.001435,-0.000241,-0.000241,-0.155298
2020-01-02 13:30:00-05:00,-0.000222,0.000496,-0.000191,0.001454,0.001454,0.107487
2020-01-02 14:30:00-05:00,0.001442,0.001049,0.001630,0.001022,0.001022,0.172749
...,...,...,...,...,...,...
2021-11-24 15:30:00-05:00,0.003280,0.000115,0.002221,0.000000,0.000000,0.379349
2021-11-26 09:30:00-05:00,-0.007932,-0.008198,-0.021246,-0.018275,-0.018275,-inf
2021-11-26 10:30:00-05:00,-0.010229,-0.009374,-0.002155,-0.004355,-0.004355,1.000000


In [189]:
rf = 0.012 / 252 / 7

In [190]:
a = SPY_hourly['Adj Close'][hourly_return.index]

In [192]:
na_indexes = a[pd.isna(a)].index

In [193]:
model = sm.OLS(hourly_return - rf, SPY_hourly['Adj Close'][hourly_return.index] - rf).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.000
Model:                            OLS   Adj. R-squared (uncentered):             -0.000
Method:                 Least Squares   F-statistic:                            0.08704
Date:                Mon, 29 Nov 2021   Prob (F-statistic):                       0.768
Time:                        03:19:58   Log-Likelihood:                          10286.
No. Observations:                2772   AIC:                                 -2.057e+04
Df Residuals:                    2771   BIC:                                 -2.056e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Adj Close      0.0087      0.030      0.295      0.768      -0.049       0.067
==============================================================================
Omnibus:                     2326.876   Durbin-Watson:                   1.396
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            39887.137
Skew:                           4.156   Prob(JB):                         0.00
Kurtosis:                      19.620   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [194]:
tr = (net_return - 0.012 / 4) / model.params[0]
tr

5774.377916123818

In [195]:
stock_df.iloc[-1] / stock_df.iloc[-1].sum()

SPY     0.249908
VOO     0.250134
SPLG     0.23997
IVV     0.259988
Name: 2021-11-26 13:00:00-05:00, dtype: object

In [196]:
totalTransactionCost

103680.93645363946